### Import library

In [1]:
import pandas as pd
from bokeh.io import output_notebook,show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,HoverTool,DatetimeTickFormatter
from bokeh.models.annotations import Arrow,Label,BoxAnnotation
from bokeh.models.arrow_heads import VeeHead

In [2]:
output_notebook()

Loading BokehJS ...

### Read dataset

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/30/21,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,118659,120216,120216,120216,120216,125937,127464,129021,130113,131586
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132521,132523,132526,132534,132535,132537,132544,132557,132565,132580
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,139626,140075,140550,141007,141471,141966,142447,143032,143652,144483
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13911,13918,13918,13918,13918,13918,13991,14021,14050,14075
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,38849,38965,39089,39172,39230,39300,39375,39491,39593,39791


In [4]:
df.shape

(279, 539)

### Wide to long dataframe

In [5]:
df_melt=pd.melt(df, id_vars=['Province/State', 'Country/Region', 'Lat', 
                             'Long'], var_name='date', 
                value_name='confirmed_case')
df_melt.head()

,Province/State,Country/Region,Lat,Long,date,confirmed_case
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [6]:
df_melt.shape

(149265, 6)

### Filter bd dataframe

In [7]:
df_bd=df_melt[df_melt["Country/Region"]=='Bangladesh']
bd=df_bd.loc[:, ["date", "confirmed_case"]]
bd['daily_case']=df_bd.confirmed_case.diff().fillna(0)

In [8]:
bd.head()

,date,confirmed_case,daily_case
20,1/22/20,0,0.0
299,1/23/20,0,0.0
578,1/24/20,0,0.0
857,1/25/20,0,0.0
1136,1/26/20,0,0.0


In [9]:
bd.tail()

,date,confirmed_case,daily_case
147890,7/5/21,954881,9964.0
148169,7/6/21,966406,11525.0
148448,7/7/21,977568,11162.0
148727,7/8/21,989219,11651.0
149006,7/9/21,1000543,11324.0


In [10]:
bd.shape

(535, 3)

In [11]:
bd.dtypes

date               object
confirmed_case      int64
daily_case        float64
dtype: object

### Convert date column to pandas datetime object

In [12]:
bd["date"]=pd.to_datetime(bd.date)

In [13]:
bd.dtypes

date              datetime64[ns]
confirmed_case             int64
daily_case               float64
dtype: object

### Pandas `DataFrame` to Bokeh `ColumnDataSource`

In [14]:
bd_source=ColumnDataSource(bd)

### Interactive plot

In [15]:
hover=HoverTool(tooltips=[("date", "@date{%F}"), 
                    ("daily case", "@daily_case")], 
formatters={"@date" : "datetime"}, mode="mouse")
            

p=figure(plot_width=600, plot_height=400, x_axis_type="datetime",
        tools=[hover, "pan", "reset", "save"], 
        title="Daily Covid-19 cases in Bangladesh")
#p.add_tools(hover)
p.line(x="date", y="daily_case", source=bd_source, 
       color="DarkBlue", 
       line_width=1.5)

# add arrow
arrow1=Arrow(x_start=pd.to_datetime("2020-03-08"),
             y_start=1000,
             x_end=pd.to_datetime("2020-03-08"), 
             y_end=3, 
             line_color="red",
             end=VeeHead(size=8,line_color="red", fill_color="red"))
p.add_layout(arrow1)

#add frist case label
label1=Label(x=pd.to_datetime("2020-01-03"),
        y=1000, 
        text_baseline="middle",
        text="First Covid-19 Cases 8 March", 
        text_color="red")
p.add_layout(label1)

#add first lockdown box
first_lockdown=BoxAnnotation(left=pd.to_datetime("2020-03-23"), 
                            right=pd.to_datetime("2020-05-30"), 
                            top=10000, 
                            bottom=0, 
                            fill_color="gray",
                            fill_alpha=0.3)
p.add_layout(first_lockdown)

lockdown_label1=Label(x=pd.to_datetime("2020-03-23"), 
                     y=6000, 
                     text="Lockdown", 
                     text_color="gray")
p.add_layout(lockdown_label1)
 
#add second lockdown annotatino
second_lockdown=BoxAnnotation(left=pd.to_datetime("2021-04-14"),
                             top=10000, 
                             bottom=0, 
                             fill_color="gray", 
                             fill_alpha=0.3)
p.add_layout(second_lockdown)
lockdown_label2=Label(x=pd.to_datetime("2021-04-14"), 
                     y=6000, 
                     text="Lockdown", 
                     text_color="gray")
p.add_layout(lockdown_label2)


p.xaxis.axis_label="Time"
p.xaxis.axis_label_text_color="DarkGreen"
p.xaxis.formatter=DatetimeTickFormatter(months="%b %y")

p.yaxis.axis_label="Daily infection"
p.yaxis.axis_label_text_color="DarkGreen"

p.title.text_font="ecotype"
p.title.text_color="DarkGreen"
p.title.text_font_size="20px"
show(p)